In [2]:
import cv2,cvzone
from cvzone.ColorModule import ColorFinder

cap = cv2.VideoCapture('./test.webm')
myColorFinder = ColorFinder(trackBar=False)
hsvVals = {'hmin': 0, 'smin': 140, 'vmin': 148, 'hmax': 7, 'smax': 255, 'vmax': 255}

while True:
    ret, img = cap.read()
    # img = cv2.imread('ball.png')

    imgColor, mask = myColorFinder.update(img, hsvVals)
    imgContours, contours = cvzone.findContours(img,mask,minArea=50)
    print(contours)
    cv2.imshow("Color Filtered Image", imgContours)

    # CRITICAL: Use waitKey(1) to allow the GUI to update
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

[{'cnt': array([[[518, 342]],

       [[518, 343]],

       [[519, 343]],

       [[520, 344]],

       [[520, 345]],

       [[520, 346]],

       [[520, 347]],

       [[519, 348]],

       [[518, 348]],

       [[518, 349]],

       [[517, 350]],

       [[516, 350]],

       [[515, 350]],

       [[514, 350]],

       [[513, 350]],

       [[512, 350]],

       [[512, 351]],

       [[511, 352]],

       [[510, 352]],

       [[509, 352]],

       [[508, 352]],

       [[507, 352]],

       [[506, 352]],

       [[506, 353]],

       [[506, 354]],

       [[506, 355]],

       [[506, 356]],

       [[506, 357]],

       [[506, 358]],

       [[506, 359]],

       [[506, 360]],

       [[507, 360]],

       [[508, 359]],

       [[509, 359]],

       [[510, 359]],

       [[511, 359]],

       [[511, 358]],

       [[512, 357]],

       [[513, 356]],

       [[514, 355]],

       [[515, 354]],

       [[515, 353]],

       [[515, 352]],

       [[516, 351]],

       [[517, 351]],

 

error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [11]:
#!/usr/bin/env python3
# subscriber.py
import cv2, zmq, numpy as np, time, threading, queue, traceback, os, sys
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        # seconds for fps moving window
DISPLAY_FPS = 20
VISUALIZE = True     # show tiled view window
MIN_BLOB_AREA = 80        # tiny specks ignored
MAX_BLOB_AREA = 20000     # huge regions ignored

def mask_orange(hsv, lab):
    h, s, v = cv2.split(hsv)
    # Hue wraps in OpenCV: orange roughly 5-22 (0..179)
    lower_h = 5; upper_h = 22
    m_h = cv2.inRange(h, lower_h, upper_h)
    m_s = cv2.inRange(s, 110, 255)   # require decent saturation
    m_v = cv2.inRange(v, 90, 255)    # reasonably bright
    return cv2.bitwise_and(cv2.bitwise_and(m_h, m_s), m_v)

def mask_purple(hsv, lab):
    # Purple/blue: hue around 120-150 in many lights but mixed lighting causes shifts.
    # Use Lab a,b clues too: purple tends to show particular (a,b) signature.
    h, s, v = cv2.split(hsv)
    l, a, b = cv2.split(lab)
    m_h = cv2.inRange(h, 120, 160)            # wide hue band
    m_s = cv2.inRange(s, 40, 255)             # allow lower sat since purple can be dark
    m_v = cv2.inRange(v, 30, 220)             # dark to mid brightness
    # Lab b-channel for purple tends to be low/negative in some lighting; accept wide band
    m_b = cv2.inRange(b, 0, 160)              # permissive
    mask = cv2.bitwise_and(m_h, m_s)
    mask = cv2.bitwise_and(mask, m_v)
    mask = cv2.bitwise_or(mask, m_b)   # include lab cue
    return mask

def mask_brown(hsv, lab):
    # Brown: low brightness, low-medium saturation, hue around 5-30 or 10-30 depending on wood floor
    h, s, v = cv2.split(hsv)
    l, a, b = cv2.split(lab)
    m_h = cv2.inRange(h, 5, 30)
    m_s = cv2.inRange(s, 30, 200)
    m_v = cv2.inRange(v, 20, 140)  # dark
    # Lab 'a' tends to be positive for brown; use permissive range
    m_a = cv2.inRange(a, 120, 200)
    mask = cv2.bitwise_and(m_h, m_s)
    mask = cv2.bitwise_and(mask, m_v)
    mask = cv2.bitwise_or(mask, m_a)
    return mask


K_OPEN = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
K_CLOSE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))

class BallDetectorThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, result_dict, lock):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.result = result_dict
        self.lock = lock
        self.stop_flag = False

    def run(self):
        print(f"[{self.cam_name}] Detector thread started.")
        while not self.stop_flag:
            try:
                frame,ts = self.frame_queue.get(timeout=0.1)
            except queue.Empty:
                continue
            try:
                H0, W0 = frame.shape[:2]
                frame_proc = frame.copy()
                hsv = cv2.cvtColor(frame_proc, cv2.COLOR_BGR2HSV)
                lab = cv2.cvtColor(frame_proc, cv2.COLOR_BGR2LAB)

                m_orange = mask_orange(hsv, lab)
                m_purple = mask_purple(hsv, lab)
                m_brown = mask_brown(hsv, lab)

                mask = cv2.bitwise_or(m_orange, m_purple)
                mask = cv2.bitwise_or(mask, m_brown)

                # morphology
                mask = cv2.medianBlur(mask, 5)
                mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, K_OPEN)
                mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, K_CLOSE)

                # find contours
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                best = None
                best_score = 0.0

                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    if area < MIN_BLOB_AREA or area > MAX_BLOB_AREA:
                        continue
                    # circularity measure
                    circ = circularity(cnt)
                    # bounding rect aspect ratio
                    x,y,w,h = cv2.boundingRect(cnt)
                    aspect = w / float(h + 1e-6)
                    # prefer blobs that are fairly circular and reasonably sized
                    score = circ * (area / float(MAX_BLOB_AREA))
                    # slightly prefer near-square shapes
                    if 0.6 <= aspect <= 1.6:
                        score *= 1.2
                    if score > best_score:
                        best_score = score
                        best = (cnt, area, circ, (x,y,w,h))

                detection = None
                if best is not None:
                    cnt, area, circ, (x,y,w,h) = best
                    # compute centroid in original image coords
                    M = cv2.moments(cnt)
                    if M["m00"] != 0:
                        cx = int(M["m10"]/M["m00"])
                        cy = int(M["m01"]/M["m00"])
                    else:
                        cx, cy = x + w//2, y + h//2

                    # classification: compute mean HSV inside mask region (on proc frame)
                    mask_small = np.zeros(frame_proc.shape[:2], dtype=np.uint8)
                    cv2.drawContours(mask_small, [cnt], -1, 255, -1)
                    # mean in HSV and Lab
                    mean_hsv = cv2.mean(hsv, mask=mask_small)[:3]
                    mean_lab = cv2.mean(lab, mask=mask_small)[:3]

                    # decide label by simple rules (robust in practice)
                    label = "unknown"
                    # check orange first (saturation and hue)
                    Hm, Sm, Vm = mean_hsv
                    Lm, Am, Bm = mean_lab
                    if 5 <= Hm <= 30 and Sm >= 110 and Vm >= 80:
                        label = "orange"
                    else:
                        # purple heuristic: hue in 120-160 OR lab a/b signature
                        if (120 <= Hm <= 160) or (Am < 140 and Bm < 145):
                            label = "purple"
                        else:
                            # brown heuristic: dark (V small), lab a higher
                            if Vm < 140 and Am >= 140:
                                label = "brown"
                            else:
                                # fallback by comparing color distance to prototypes
                                # prototypes in LAB (approx): orange, purple, brown
                                prot_orange = np.array([180, 140, 140])  # approximate
                                prot_purple = np.array([110, 140, 130])
                                prot_brown = np.array([120, 150, 140])
                                d_or = np.linalg.norm(np.array(mean_lab) - prot_orange)
                                d_pu = np.linalg.norm(np.array(mean_lab) - prot_purple)
                                d_br = np.linalg.norm(np.array(mean_lab) - prot_brown)
                                dmin = min(d_or, d_pu, d_br)
                                if dmin == d_or:
                                    label = "orange"
                                elif dmin == d_pu:
                                    label = "purple"
                                else:
                                    label = "brown"

                    detection = {
                        "bbox": (x,y,w,h),
                        "centroid": (cx,cy),
                        "area": int(area),
                        "circularity": float(circ),
                        "label": label,
                        "ts": ts,
                        "proc_time": time.time()
                    }

                # save latest detection
                with self.lock:
                    self.result[self.cam_name] = detection

            except Exception as e:
                print(f"[ERROR-{self.cam_name}] detector exception:", e)
                traceback.print_exc()

        print(f"[DETECT-{self.cam_name}] Detector thread stopped")

    def stop(self):
        self.stop_flag = True

# ---------- Helpers ----------
def fmt_ts(ts):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

def update_fps(camera, cam_ts):
    dq = fps_windows[camera]
    dq.append(cam_ts)
    # pop older than window
    while dq and (cam_ts - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps

def circularity(contour):
    area = cv2.contourArea(contour)
    perim = cv2.arcLength(contour, True)
    if perim <= 0: return 0.0
    return 4.0 * np.pi * area / (perim*perim)


# ---------- ZMQ subscriber ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 1)
sub.setsockopt(zmq.CONFLATE, 1)  # keep only last message
sub.setsockopt(zmq.LINGER, 0)


# ---- ACTIVE FLUSH ----
flushed = 0
while True:
    try:
        sub.recv_multipart(flags=zmq.NOBLOCK)
        flushed += 1
    except zmq.Again:
        break
if flushed > 0:
    print(f"[Subscriber] Flushed {flushed} stale messages.")
for t in SUB_TOPICS:
    sub.setsockopt(zmq.SUBSCRIBE, t)

# per-camera state
frames = {}
fps_windows = defaultdict(lambda: deque())   # deque of capture times
frame_queues = {t.decode(): queue.Queue(maxsize=1) for t in SUB_TOPICS}
detect_results = {}
detect_lock = threading.Lock()
det_threads = {}

for t in SUB_TOPICS:
    cam_name = t.decode()
    dt = BallDetectorThread(cam_name, frame_queues[cam_name], detect_results, detect_lock)
    dt.start()
    det_threads[cam_name] = dt


print("[Subscriber] connected, waiting for frames... (Press ESC to exit)")

last_show = time.time()
# ---------- Main loop ----------
try:
    while True:
        parts = recv_latest(sub)
        if parts is None:
            if VISUALIZE:
                cv2.waitKey(1)
            continue

        # unpack message
        topic = parts[0]
        cam = topic.decode()

        if len(parts) >= 3:
            ts_part = parts[1]
            jpg_part = parts[2]
        else:
            ts_part = None
            jpg_part = parts[1]

        recv_time = time.time()

        try:
            cam_ts = float(ts_part.decode()) if ts_part else recv_time
        except:
            cam_ts = recv_time

        img = cv2.imdecode(np.frombuffer(jpg_part, np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            continue

        fps = update_fps(cam, cam_ts)

        frames[cam] = {
            "img": img,
            "cam_ts": cam_ts,
            "fps": fps,
        }

        fq = frame_queues[cam]
        try:
            fq.get_nowait()  # clear old
        except queue.Empty:
            pass
        try:
            fq.put_nowait((img.copy(), cam_ts))
        except queue.Full:
            pass

        # If we have at least 2 cameras, compute drift and show tiled view
        if all(k in frames for k in [t.decode() for t in SUB_TOPICS]):
            cams = [t.decode() for t in SUB_TOPICS]
            left = frames[cams[0]]
            right = frames[cams[1]]

            # compute drift in ms between corrected timestamps
            drift_ms = abs(left["cam_ts"] - right["cam_ts"]) * 1000.0

            # overlay text on each image
            def overlay(info, cam_name):
                im = info["img"].copy()
                y = 20
                cv2.putText(im, f"{cam_name}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (199,32,20), 2)
                cv2.putText(im, f"FPS: {info['fps']:.1f}", (10, y+26), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (14,117,5), 2)
                cv2.putText(im, f"cam_ts: {fmt_ts(info['cam_ts'])}", (10, y+52), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (5,12,117), 1)

                # draw detection if exists
                with detect_lock:
                    d = detect_results.get(cam_name, None)
                    if d is not None:
                        bbox = d["bbox"]
                        cx,cy = d["centroid"]
                        label = d["label"]
                        area = d["area"]
                        circ = d["circularity"]
                        # bbox draw
                        x,y,w,h = bbox
                        cv2.rectangle(im, (x,y), (x+w, y+h), (0,255,255), 2)
                        cv2.circle(im, (cx,cy), 3, (0,0,255), -1)
                        cv2.putText(im, f"{label} {area}px", (x, y-8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,0), 1)
                        cv2.putText(im, f"circ:{circ:.2f}", (x, y+h+16), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (200,200,0), 1)
                return im

            if VISUALIZE and time.time()-last_show > 1.0/DISPLAY_FPS:
                left_im = overlay(left, cams[0])
                right_im = overlay(right, cams[1])

                # resize to same height and tile horizontally
                h = max(left_im.shape[0], right_im.shape[0])
                right_resized = cv2.resize(right_im, (left_im.shape[1], h))
                tile = np.hstack([left_im, right_resized])

                # Draw drift and timestamp summary on top-left of tiled image
                cv2.putText(tile, f"Drift: {drift_ms:.1f} ms", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
                cv2.putText(tile, f"Host now: {fmt_ts(time.time())}", (10, 44), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

                last_show = time.time()
                cv2.imshow("Both Cameras (tiled)", tile)
            elif not VISUALIZE:
                with detect_lock:
                    s0 = detect_results.get(cams[0], None)
                    s1 = detect_results.get(cams[1], None)
                    t0 = s0["label"] if s0 else "none"
                    t1 = s1["label"] if s1 else "none"
                status = (f"Drift {drift_ms:.1f} ms | {cams[0]} FPS:{left['fps']:.1f} | "
                          f"{cams[1]} FPS:{right['fps']:.1f} | Labels: {cams[0]}:{t0} {cams[1]}:{t1}")
                sys.stdout.write("\r" + status + " " * 20)
                sys.stdout.flush()
            

        if cv2.waitKey(1) & 0xFF == 27:
            break

except KeyboardInterrupt:
    pass
finally:
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()
    print("\n[Subscriber] exited cleanly.")


[kreo1] Detector thread started.
[kreo2] Detector thread started.
[Subscriber] connected, waiting for frames... (Press ESC to exit)

[Subscriber] exited cleanly.
